- 변경사항
  - koELECTRA 모델 사용
  - 모델에 맞게 코드 수정

In [2]:
import random
import torch
from transformers import ElectraTokenizerFast, ElectraForTokenClassification
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

def generate_fake_data(num_samples):
    names = ["홍길동", "김철수", "이영희", "박지성", "최민수", "정수연", "강태풍", "윤미래", "송하늘", "조해성"]
    places = ["서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "제주", "수원", "창원"]
    organizations = ["삼성", "LG", "현대", "SK", "롯데", "네이버", "카카오", "쿠팡", "배달의민족", "토스"]
    phone_prefixes = ["010", "011", "016", "017", "018", "019"]
    rrn_prefixes = ["990101", "850505", "770303", "880808", "920414", "660606", "950707", "001212"]
    bank_names = ["국민은행", "신한은행", "우리은행", "하나은행", "기업은행", "농협은행", "케이뱅크", "카카오뱅크"]
    job_titles = ["사원", "대리", "과장", "차장", "부장", "이사", "상무", "전무", "대표", "인턴"]
    departments = ["영업부", "마케팅부", "인사부", "재무부", "개발부", "디자인부", "고객서비스팀", "연구소"]

    data = []
    for _ in range(num_samples):
        name = random.choice(names)
        place = random.choice(places)
        org = random.choice(organizations)
        phone = f"{random.choice(phone_prefixes)}-{random.randint(1000, 9999)}-{random.randint(1000, 9999)}"
        rrn = f"{random.choice(rrn_prefixes)}-{random.randint(1000000, 9999999)}"
        bank = random.choice(bank_names)
        account = f"{random.randint(100, 999)}-{random.randint(10000, 99999)}-{random.randint(10000, 99999)}"
        job_title = random.choice(job_titles)
        department = random.choice(departments)

        # 다양한 구조의 문장 중 하나를 무작위로 선택하여 가상의 데이터를 생성
        structures = [
            lambda: f"{name}의 인적사항: 거주지 {place}, 연락처 {phone}, 주민등록번호 {rrn}. {org} {department} {job_title}로 근무 중. {bank} 계좌번호: {account}",
            lambda: f"{org} {department}의 {job_title} {name}씨 정보 - 전화번호: {phone}, 주소지: {place}, 주민번호: {rrn}, {bank} 계좌: {account}",
            lambda: f"이름: {name}, 직장: {org} ({department}, {job_title}), 연락처: {phone}, 거주지: {place}, 주민등록번호: {rrn}, 급여계좌: {bank} {account}",
            lambda: f"{place}에 거주하는 {name}({rrn})씨는 {org} {department}에서 {job_title}으로 일하고 있습니다. 연락처는 {phone}이며, {bank} 계좌({account})를 사용합니다.",
            lambda: f"{bank} 고객정보 - 성명: {name}, 계좌번호: {account}, 전화번호: {phone}, 직장: {org} {department} ({job_title}), 주민등록번호: {rrn}, 주소: {place}",
            lambda: f"[인사기록] 직원명: {name}, 부서: {department}, 직위: {job_title}, 주민번호: {rrn}, 연락처: {phone}, 주소: {place}, 급여계좌: {bank} {account}",
            lambda: f"{org} {department} {name} {job_title}님의 연락처는 {phone}입니다. 주민등록번호 {rrn}, {place} 거주, {bank} 계좌번호 {account}.",
            lambda: f"성명 {name} (주민등록번호: {rrn}) / 근무처: {org} {department} / 직위: {job_title} / 주소: {place} / 연락처: {phone} / 계좌: {bank} {account}",
            lambda: f"{place}의 {name}씨는 {org} {department}에서 {job_title}으로 일합니다. 연락처 {phone}, 주민번호 {rrn}, {bank} 계좌 {account}를 사용 중입니다.",
            lambda: f"직원정보: {name} / {org} / {department} / {job_title} / {place} 거주 / {phone} / {rrn} / {bank} {account}",
            lambda: f"{bank} {account} 계좌의 예금주 {name}님 정보 - 주소: {place}, 전화: {phone}, 주민등록번호: {rrn}, 직장: {org} {department} {job_title}",
            lambda: f"{name}씨는 {org}의 {job_title}입니다. {department} 소속이며, {place}에 삽니다. 연락처는 {phone}, 주민번호는 {rrn}, {bank} 계좌는 {account}입니다.",
            lambda: f"[{org} 직원 데이터] 이름: {name}, 부서: {department}, 직책: {job_title}, 전화번호: {phone}, 주소: {place}, 주민등록번호: {rrn}, 급여계좌: {bank} {account}",
            lambda: f"주민등록번호 {rrn}의 {name}님은 {org} {department} {job_title}입니다. {place}에 거주 중이며, 연락처는 {phone}, {bank} 계좌번호는 {account}입니다.",
            lambda: f"{place} 주민 {name}의 신상정보 - 직장: {org} {department}, 직위: {job_title}, 전화번호: {phone}, 주민번호: {rrn}, 계좌정보: {bank} {account}",
            lambda: f"{org} {department}의 {name} {job_title}에 대한 정보입니다. 주소: {place}, 전화: {phone}, 주민등록번호: {rrn}, {bank} 계좌번호: {account}",
            lambda: f"이름: {name} / 주민번호: {rrn} / 주소: {place} / 직장: {org} / 부서: {department} / 직위: {job_title} / 연락처: {phone} / 계좌: {bank} {account}",
            lambda: f"{bank} 고객 {name}님 ({rrn})의 계좌번호는 {account}입니다. {org} {department} {job_title}이며, {place}에 거주합니다. 연락처: {phone}",
            lambda: f"{name} ({org} {department}, {job_title}) - 주소: {place}, 전화번호: {phone}, 주민등록번호: {rrn}, {bank} 계좌: {account}",
            lambda: f"[개인정보] 성명: {name}, 주민번호: {rrn}, 근무지: {org} {department}, 직책: {job_title}, 거주지: {place}, 연락처: {phone}, 계좌: {bank} {account}"
        ]

        sample = random.choice(structures)()
        data.append(sample)
    return data


def tokenize_and_label(text):
    tokens = []
    labels = []
    words = text.split()
    for word in words:
        if "-" in word:
            parts = word.split("-")
            if len(parts) == 3 and parts[0] in ["010", "011", "016", "017", "018", "019"]:
                tokens.extend(parts)
                labels.extend(["B-PH", "I-PH", "I-PH"])
            elif len(parts) == 2 and len(parts[0]) == 6 and parts[0].isdigit():
                tokens.extend(parts)
                labels.extend(["B-RRN", "I-RRN"])
            elif len(parts) == 3 and len(parts[0]) == 3 and all(part.isdigit() for part in parts):
                tokens.extend(parts)
                labels.extend(["B-ACC", "I-ACC", "I-ACC"])
            else:
                tokens.append(word)
                labels.append("O")  # 개인 정보가 아닌 경우 'O' 라벨 부여
        else:
            tokens.append(word)
            labels.append("O")
    return tokens, labels

def load_data_with_new_labels(file_path, new_data):
    texts, labels = [], []
    label_map = {
        'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-LC': 3, 'I-LC': 4,
        'B-OG': 5, 'I-OG': 6, 'B-QT': 7, 'I-QT': 8, 'B-DT': 9,
        'I-DT': 10, 'B-TI': 11, 'I-TI': 12, 'B-LT': 13, 'I-LT': 14,
        'B-PO': 15, 'I-PO': 16, 'B-PH': 17, 'I-PH': 18, 'B-RRN': 19, 'I-RRN': 20,
        'B-ACC': 21, 'I-ACC': 22  # 새로 추가된 계좌번호 라벨
    }

    with open(file_path, 'r', encoding='utf-8') as file:
        text, label = [], []
        for line in file:
            if line.startswith('##') or not line.strip():
                if text:
                    texts.append(" ".join(text))
                    labels.append([label_map[l] for l in label])
                    text, label = [], []
                continue
            parts = line.strip().split('\t')
            if len(parts) != 2:
                continue
            char, tag = parts
            text.append(char)
            label.append(tag)

    # 새로 생성한 가상 데이터셋 추가
    for sample in new_data:
        tokens, tags = tokenize_and_label(sample)
        texts.append(" ".join(tokens))
        labels.append([label_map[tag] for tag in tags])

    return texts, labels

# 가상 데이터셋 생성
fake_data = generate_fake_data(2000)
# 기존 데이터에 가상 데이터셋을 추가하여 로드
train_texts, train_labels = load_data_with_new_labels('klue-ner-v1.1_train (2).tsv', fake_data)
dev_texts, dev_labels = load_data_with_new_labels('klue-ner-v1.1_dev (3).tsv', fake_data)

# 학습 데이터셋을 정의하는 클래스
class PIIDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        labels = self.labels[item]

        encoding = self.tokenizer(
            text.split(),  # 단어 단위로 분리하여 토크나이징
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            is_split_into_words=True,  # 단어 단위로 토크나이징
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        aligned_labels = [-100] * len(input_ids)  # 패딩된 토큰은 -100으로 마스킹
        word_ids = encoding.word_ids()  # 원본 텍스트의 단어 위치

        for i, word_id in enumerate(word_ids):
            if word_id is None:
                continue
            aligned_labels[i] = labels[word_id]  # 원본 단어의 라벨을 토큰에 매핑

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(aligned_labels, dtype=torch.long)
        }


# "Fast" 버전의 토크나이저로 변경
tokenizer = ElectraTokenizerFast.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraForTokenClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=23)

max_len = 128

train_dataset = PIIDataset(train_texts, train_labels, tokenizer, max_len)
dev_dataset = PIIDataset(dev_texts, dev_labels, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Early Stopping 설정 (성능이 개선되지 않으면 학습 중단)
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    callbacks=[early_stopping_callback]
)

trainer.train()
trainer.evaluate()

# 모델과 토크나이저 저장
model.save_pretrained('./pii_masking_model')
tokenizer.save_pretrained('./pii_masking_model')

print("Training completed and model saved.")


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.198600,0.180708
2,0.136400,0.126268
3,0.095600,0.119148
4,0.088200,0.111439
5,0.080100,0.110998


Training completed and model saved.


In [3]:
def detect_and_mask_pii(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    label_map = {
        0: 'O', 1: 'B-PS', 2: 'I-PS', 3: 'B-LC', 4: 'I-LC',
        5: 'B-OG', 6: 'I-OG', 7: 'B-QT', 8: 'I-QT', 9: 'B-DT',
        10: 'I-DT', 11: 'B-TI', 12: 'I-TI', 13: 'B-LT', 14: 'I-LT',
        15: 'B-PO', 16: 'I-PO', 17: 'B-PH', 18: 'I-PH', 19: 'B-RRN', 20: 'I-RRN',
        21: 'B-ACC', 22: 'I-ACC'  # Added new labels for account numbers
    }

    masked_text = []
    current_mask = None
    for token, pred in zip(tokens, predictions[0]):
        if token in ["[CLS]", "[SEP]", "[PAD]"]:
            continue
        label = label_map[pred.item()]
        if label.startswith('B-'):
            current_mask = label[2:]  # Extract entity type (PS, LC, OG, etc.)
            masked_text.append(f'[MASK-{current_mask}]')
        elif label.startswith('I-'):
            if current_mask != label[2:]:
                current_mask = label[2:]
                masked_text.append(f'[MASK-{current_mask}]')
        else:
            if token.startswith("##"):
                masked_text[-1] += token[2:]
            else:
                masked_text.append(token)
            current_mask = None

    masked_text = " ".join(masked_text)
    masked_text = masked_text.replace(" ##", "").replace(" .", ".").strip()
    return masked_text

In [4]:
test_text = "강감찬 장군은 고려시대 인물로 , 현재 서울시 관악구 봉천동 1234번지에 그의 동상이 있습니다 . 그의 생년은 948년으로 알려져 있습니다 ."
masked_text = detect_and_mask_pii(test_text, model, tokenizer, device)
print(masked_text)

[MASK-PS] 장군은 고려시대 인물로 , 현재 서울시 [MASK-LC] 1234번지에 그의 동상이 있습니다. 그의 생년은 [MASK-TI] [MASK-DT]으로 알려져 있습니다.


In [5]:
test_text = "이순신의 전화번호는 010-3318-3994이다. 그의 주민등록번호는 000908-2133434이다."
masked_text = detect_and_mask_pii(test_text, model, tokenizer, device)
print(masked_text)

이순신의 전화번호는 010 - 3318 - 3994이다. 그의 주민등록번호는 000908 - 2133434이다.


In [6]:
test_text = "2012년에 태어난 사촌 조카는 서울시 성동구에서 유치원을 다니고 있다. 조카는 우리은행 100-232344-433221 계좌를 사용한다."
masked_text = detect_and_mask_pii(test_text, model, tokenizer, device)
print(masked_text)

[MASK-DT]에 태어난 사촌 조카는 서울시 [MASK-LC]에서 유치원을 다니고 있다. 조카는 우리은행 100 - 232344 - 433221 계좌를 사용한다.
